In [4]:
from PIL import Image
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline

In [3]:
path_to_dataset = '../data/dataset/'
train_images_folder = os.path.join(path_to_dataset, 'train_images')
train_table_path = os.path.join(path_to_dataset, 'train.csv')

In [25]:
train_table = pd.read_csv(train_table_path).fillna(-1)

In [28]:
train_table.head(15)

,ImageId_ClassId,EncodedPixels
0,0002cc93b.jpg_1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0002cc93b.jpg_2,-1
2,0002cc93b.jpg_3,-1
3,0002cc93b.jpg_4,-1
4,00031f466.jpg_1,-1
5,00031f466.jpg_2,-1
6,00031f466.jpg_3,-1
7,00031f466.jpg_4,-1
8,000418bfc.jpg_1,-1
9,000418bfc.jpg_2,-1


In [39]:
def rleToMask(rleString,height,width):
    rows, cols = height, width
    if rleString == -1:
        return np.zeros((height, width))
    else:
        rleNumbers = [int(numstring) for numstring in rleString.split(' ')]
        rlePairs = np.array(rleNumbers).reshape(-1,2)
        img = np.zeros(rows*cols,dtype=np.uint8)
        for index,length in rlePairs:
            index -= 1
            img[index:index+length] = 255
            img = img.reshape(cols,rows)
            img = img.T
        return img

In [52]:
channels_data = {}
for img_id_class, data in train_table.values:
    img_id, channel_class = img_id_class.split('_')
    
    if img_id not in channels_data.keys():
        channels_data[img_id] = {}
    
    channels_data[img_id][int(channel_class)] = data

In [55]:
images_names_list = list(set(map(lambda x: x.split('_')[0], train_table.values[:, 0])))
i = 0
images_names_list[i], channels_data[images_names_list[i]]

('f3e1e7c47.jpg',
 {1: -1,
  2: '231682 8 231938 22 232194 36 232450 51 232706 65 232858 103 232962 79 233114 103 233218 93 233370 205 233626 205 233882 204 234138 204 234394 204 234650 203 234906 103 235016 93 235162 77 235286 79 235418 26 235556 64 235826 50 236096 36 236366 21 236636 7',
  3: -1,
  4: -1})